### Je m'amuse sur des trucs pas amusants

In [40]:
import pandas as pd
import geopandas as gpd
import xarray as xr
import glob
import matplotlib.pyplot as plt


In [ ]:
path = "C:/Users/colin/Downloads/donnee*.parquet"  # Update with your own path
print(glob.glob(path))
files = glob.glob(path)
df = pd.read_parquet(files[0]) 

df.indicateur.value_counts().sort_index()

df_temp = df[df.CODGEO_2024=='34199']
#make a plot for each of the indicateur of the temporal evolution
import ipywidgets as widgets
from ipywidgets import interact

def plot_variable(indicateur):
    plt.figure(figsize=(10, 6))
    for ind in df_temp.indicateur.unique():
        if ind == indicateur:
            df_ind = df_temp[df_temp.indicateur == ind]
            plt.plot(df_ind.annee, df_ind.taux_pour_mille, label=ind)
    plt.xlabel('Année')
    plt.ylabel('Taux pour mille')
    plt.title(f"Évolution temporelle : {indicateur}")
    plt.legend()
    plt.show()

interact(plot_variable, indicateur=widgets.Dropdown(options=sorted(df_temp.indicateur.unique())))


['C:/Users/colin/Downloads\\donnee-comm-data.gouv-parquet-2024-geographie2024-produit-le2025-03-14.parquet']


interactive(children=(Dropdown(description='indicateur', options=('Autres coups et blessures volontaires', 'Ca…

<function __main__.plot_variable(indicateur)>

In [37]:
path = 'C:/Users/colin/Downloads/temp_agg.nc'  # Update with your own path
print(glob.glob(path))
ds = xr.open_dataset(path) 
ds.where(ds.LAU_NAME=='Pézenas', drop=True)


['C:/Users/colin/Downloads/temp_agg.nc']


<xarray.Dataset>
Dimensions:           (poly_idx: 1, year: 10, month: 12)
Coordinates:
  * poly_idx          (poly_idx) int32 32330
  * year              (year) int64 2015 2016 2017 2018 ... 2021 2022 2023 2024
  * month             (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    temperature       (poly_idx, year, month) float64 8.197 6.911 ... 8.081
    positive_anomaly  (poly_idx, year, month) float64 0.0 0.0 0.0 ... 0.0 0.0
    LAU_ID            (poly_idx) object '34199'
    LAU_NAME          (poly_idx) object 'Pézenas'

In [35]:
path = "C:/Users/colin/Downloads/donnee*.parquet"  # Update with your own path
print(glob.glob(path))
files = glob.glob(path)
df = pd.read_parquet(files[0])  # Read the first file in the list
# df = df[df.indicateur == 'Coups et blessures volontaires']
# df = df[df.indicateur == 'Violences sexuelles']
# df = df[df.indicateur == 'Destructions et dégradations volontaires']
# df = df[df.indicateur == 'Cambriolages de logement']
df = df[df.indicateur == 'Autres coups et blessures volontaires']

path = 'C:/Users/colin/Downloads/temp_agg.nc'  # Update with your own path
print(glob.glob(path))
ds = xr.open_dataset(path)  # Open the NetCDF file
df = df[['CODGEO_2024', 'annee', 'taux_pour_mille']]
summer_ano = ds.sel(month=[6,7,8]).sum(dim='month')  # Sum over the summer months (June, July, August)
summer_ano = summer_ano.to_dataframe().reset_index()

df = df.merge(summer_ano, left_on=['CODGEO_2024', 'annee'], right_on=['LAU_ID', 'year'], how='left')

from linearmodels.panel import PanelOLS
import pandas as pd

# Ensure index is MultiIndex: (entity, time)
df = df.set_index(['CODGEO_2024', 'annee'])

# Fit the model with fixed effects
mod = PanelOLS.from_formula('taux_pour_mille ~ positive_anomaly + EntityEffects + TimeEffects', data=df)
res = mod.fit(cov_type='clustered', cluster_entity=True)
print(res.summary)

['C:/Users/colin/Downloads\\donnee-comm-data.gouv-parquet-2024-geographie2024-produit-le2025-03-14.parquet']
['C:/Users/colin/Downloads/temp_agg.nc']


c:\Users\colin\anaconda3\envs\xarray_env\Lib\site-packages\linearmodels\panel\model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


                          PanelOLS Estimation Summary                           
Dep. Variable:        taux_pour_mille   R-squared:                     5.932e-05
Estimator:                   PanelOLS   R-squared (Between):              0.0013
No. Observations:              173003   R-squared (Within):               0.0004
Date:                Tue, Jul 01 2025   R-squared (Overall):              0.0014
Time:                        20:44:22   Log-likelihood                 -3.61e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      8.5833
Entities:                       28301   P-value                           0.0034
Avg Obs:                       6.1130   Distribution:                F(1,144693)
Min Obs:                       1.0000                                           
Max Obs:                       9.0000   F-statistic (robust):             6.9484
                            